<a href="https://colab.research.google.com/github/ananyakaligal/My-Machine-Learning-Journey/blob/main/CNN_MNIST%40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import time

In [40]:
# Define constants
batchSize = 16
numEpochs = 5
trainToLoss = 0.0001
bsr = 0.25
lrr = 1.0

In [5]:
def get_mnist_data(train_samples=None, test_samples=None):
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Normalize the pixel values to be between 0 and 1
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Flatten the images
    x_train = x_train.reshape(-1, 28,28,1)
    x_test = x_test.reshape(-1, 28 ,28, 1)

    # Convert labels to one-hot encoding
    y_train_one_hot = to_categorical(y_train, 10)
    y_test_one_hot = to_categorical(y_test, 10)

    # Select a subset of the data
    x_train_subset = x_train[:2000]
    y_train_one_hot_subset = y_train_one_hot[:2000]
    x_test_subset = x_test[:250]
    y_test_one_hot_subset = y_test_one_hot[:250]

    return (x_train_subset.flatten().tolist(), y_train_one_hot_subset.flatten().tolist(),
            x_test_subset.flatten().tolist(), y_test_one_hot_subset.flatten().tolist())

In [6]:
# Example usage
x_train_flattened, y_train_flattened, x_test_flattened, y_test_flattened = get_mnist_data()
print(f'x_train_flattened length: {len(x_train_flattened)}')
print(f'y_train_flattened length: {len(y_train_flattened)}')
print(f'x_test_flattened length: {len(x_test_flattened)}')
print(f'y_test_flattened length: {len(y_test_flattened)}')


11490434/11490434 [==============================] - 0s 0us/step
x_train_flattened length: 1568000
y_train_flattened length: 20000
x_test_flattened length: 196000
y_test_flattened length: 2500


In [7]:
# Print samples from the flattened data
print("Sample of x_train_flattened:", x_train_flattened[:100])
print("Sample of y_train_flattened:", y_train_flattened[:100])

Sample of x_train_flattened: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Sample of y_train_flattened: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [8]:
def format_data(data, batch_size, height, width):
    indices = []
    values = []

    # Convert the data list to a NumPy array if not already, in ecl you create a record
    data = np.array(data)

    # Calculate expected length
    expected_length = batch_size * height * width
    if data.size != expected_length:
        raise ValueError("Data length does not match expected dimensions")

    # Iterate through each dimension
    for i in range(batch_size):
        for row in range(height):
            for col in range(width):
                index = (i * height * width) + (row * width + col)
                indices.append([i+1, row+1, col+1])  #ECL indicing starts from 1
                values.append(float(data[index]))

    return np.array(indices), np.array(values)

In [20]:
# Formatting data
formatted_x_train_indices, formatted_x_train_values = format_data(x_train_flattened,2000,28,28)
formatted_x_test_indices, formatted_x_test_values = format_data(x_test_flattened,250,28,28)

In [21]:
 # Check shapes
print("Shape of formatted_x_train_indices:", formatted_x_train_indices.shape)
print("Shape of formatted_x_test_values:", formatted_x_test_values.shape)

Shape of formatted_x_train_indices: (1568000, 3)
Shape of formatted_x_test_values: (196000,)


In [22]:
# Define a function to print samples of the formatted data
def print_sample(indices, values, sample_size=30):
    for i in range(sample_size):
        print(f"Indices: {indices[i]}, Value: {values[i]}")

# Print samples
print("Sample of train_indices and train_values:")
print_sample(formatted_x_train_indices, formatted_x_train_values)

Sample of train_indices and train_values:
Indices: [1 1 1], Value: 0.0
Indices: [1 1 2], Value: 0.0
Indices: [1 1 3], Value: 0.0
Indices: [1 1 4], Value: 0.0
Indices: [1 1 5], Value: 0.0
Indices: [1 1 6], Value: 0.0
Indices: [1 1 7], Value: 0.0
Indices: [1 1 8], Value: 0.0
Indices: [1 1 9], Value: 0.0
Indices: [ 1  1 10], Value: 0.0
Indices: [ 1  1 11], Value: 0.0
Indices: [ 1  1 12], Value: 0.0
Indices: [ 1  1 13], Value: 0.0
Indices: [ 1  1 14], Value: 0.0
Indices: [ 1  1 15], Value: 0.0
Indices: [ 1  1 16], Value: 0.0
Indices: [ 1  1 17], Value: 0.0
Indices: [ 1  1 18], Value: 0.0
Indices: [ 1  1 19], Value: 0.0
Indices: [ 1  1 20], Value: 0.0
Indices: [ 1  1 21], Value: 0.0
Indices: [ 1  1 22], Value: 0.0
Indices: [ 1  1 23], Value: 0.0
Indices: [ 1  1 24], Value: 0.0
Indices: [ 1  1 25], Value: 0.0
Indices: [ 1  1 26], Value: 0.0
Indices: [ 1  1 27], Value: 0.0
Indices: [ 1  1 28], Value: 0.0
Indices: [1 2 1], Value: 0.0
Indices: [1 2 2], Value: 0.0


In [23]:
def convert_to_tensor(indices, values, shape):
    """
    Convert lists of indices and values to a TensorFlow tensor.

    Parameters:
    indices (list of lists): List of indices, where each entry is [batch, row, col, channel].
    values (list): List of values corresponding to each index.
    shape (tuple): Shape of the resulting tensor.

    Returns:
    tf.Tensor: Tensor populated with values at specified indices.
    """
    # Convert indices from 1-based to 0-based
    zero_based_indices = [np.array(idx) - 1 for idx in indices]

    # Convert indices and values to sparse tensor
    indices = np.array(zero_based_indices)
    values = np.array(values)

    # Create a sparse tensor
    sparse_tensor = tf.sparse.SparseTensor(
        indices=indices,
        values=values,
        dense_shape=shape
    )

    # Convert sparse tensor to dense tensor
    dense_tensor = tf.sparse.to_dense(sparse_tensor)

    return dense_tensor

In [25]:
# Define the shape of the tensor
shape1 = (2000, 28, 28)  # Adjust according to your tensor's dimensions
shape2 = (250, 28, 28)

# Convert to tensor
X_traintensor = convert_to_tensor(formatted_x_train_indices, formatted_x_train_values, shape1)

# Convert to tensor
X_testtensor = convert_to_tensor(formatted_x_test_indices, formatted_x_test_values, shape2)

In [26]:
##Format yTrain and yTest
def format_data(data, batch_size, height, width):
    indices = []
    values = []

    # Convert the data list to a NumPy array if not already, in ecl you create a record
    data = np.array(data)

    # Calculate expected length
    expected_length = batch_size * height * width
    if data.size != expected_length:
        raise ValueError("Data length does not match expected dimensions")

    # Iterate through each dimension
    for i in range(batch_size):
        for row in range(height):
            for col in range(width):
                index = (i * height * width) + (row * width + col)
                indices.append([i+1, row+1, col+1])  #ECL indicing starts from 1
                values.append(float(data[index]))

    return np.array(indices), np.array(values)

In [27]:

def format_data_Y(data):
    """
    Convert 1-dimensional data to indices and values.

    Parameters:
    data (list or np.array): 1D array of data values.

    Returns:
    np.array: Array of indices starting from 1.
    np.array: Array of values.
    """
    # Ensure the data is a NumPy array
    data = np.array(data)

    # Create indices starting from 1
    indices = np.arange(1, len(data) + 1)

    # Convert data values to float
    values = data.astype(float)

    return indices, values

formatted_y_train_indices, formatted_y_train_values = format_data_Y(y_train_flattened)
formatted_y_test_indices, formatted_y_test_values = format_data_Y(y_test_flattened)

# Print a sample
print("Sample indices (1-based):", formatted_y_train_indices[:10])
print("Sample values:", formatted_y_train_values[:10])

Sample indices (1-based): [ 1  2  3  4  5  6  7  8  9 10]
Sample values: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [36]:
def convert_to_tensorY(indices, values, shape):
    """
    Convert lists of 1D indices and values to a TensorFlow tensor with shape (x, y).

    Parameters:
    indices (list): List of 1D indices starting from 1.
    values (list): List of values corresponding to each index.
    shape (tuple): Shape of the resulting tensor (x, y).

    Returns:
    tf.Tensor: Tensor populated with values at specified indices.
    """
    rows, cols = shape

    # Convert 1-based indices to 0-based indices
    zero_based_indices = [(index // cols, index % cols) for index in (i - 1 for i in indices)]

    # Check for out-of-bounds indices
    for idx in zero_based_indices:
        if not (0 <= idx[0] < rows and 0 <= idx[1] < cols):
            raise ValueError(f"Index {idx} is out of bounds for shape {shape}")

    # Convert 2D indices to numpy array
    indices_np = np.array(zero_based_indices)
    values_np = np.array(values)

    # Create a sparse tensor
    sparse_tensor = tf.sparse.SparseTensor(
        indices=indices_np,
        values=values_np,
        dense_shape=shape
    )

    # Convert sparse tensor to dense tensor
    dense_tensor = tf.sparse.to_dense(sparse_tensor)

    return dense_tensor

In [37]:
# Define the shape of the tensor
shapeY1 = (2000, 10)  # Adjust according to your tensor's dimensions
shapeY2 = (250, 10)

# Convert to tensor
Y_traintensor = convert_to_tensorY(formatted_y_train_indices, formatted_y_train_values, shapeY1)

# Convert to tensor
Y_testtensor = convert_to_tensorY(formatted_y_test_indices, formatted_y_test_values, shapeY2)

In [38]:
# Define the CNN model
def create_cnn_model():
    model = models.Sequential()

    # Add layers according to the provided architecture
    model.add(layers.Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.Flatten())

    model.add(layers.Dense(256))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(10))
    model.add(layers.Activation('softmax'))

    return model


In [39]:
# Create and compile the model
model = create_cnn_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [41]:
# Train the model
history = model.fit(X_traintensor, Y_traintensor,
                    batch_size=batchSize,
                    epochs=numEpochs,
                    verbose=1)

Epoch 1/5
125/125 [==============================] - 7s 29ms/step - loss: 0.6837 - categorical_accuracy: 0.8200
Epoch 2/5
125/125 [==============================] - 4s 30ms/step - loss: 0.2341 - categorical_accuracy: 0.9485
Epoch 3/5
125/125 [==============================] - 5s 36ms/step - loss: 0.1538 - categorical_accuracy: 0.9605
Epoch 4/5
125/125 [==============================] - 4s 36ms/step - loss: 0.1394 - categorical_accuracy: 0.9645
Epoch 5/5
125/125 [==============================] - 6s 49ms/step - loss: 0.1120 - categorical_accuracy: 0.9715


In [42]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_testtensor, Y_testtensor, verbose=2)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")

8/8 - 0s - loss: 0.1394 - categorical_accuracy: 0.9560 - 408ms/epoch - 51ms/step
Test loss: 0.13939395546913147
Test accuracy: 0.9559999704360962
